In [1]:
import xarray as xr
import glob

from dask.distributed import Client, LocalCluster
from dask_jobqueue import SLURMCluster

# ERA5 Land data: from NetCDF3 to Zarr

In this notebook we load ERA5 Land data stored in a collection of NetCDF3 files and we save it as a (chunked) Zarr store. It is global.

## Input variables

In [2]:
year = 2012

In [3]:
ROOT_DIR = '/gpfs/work2/0/ttse0619'
ERA5_DIR = (
    f'{ROOT_DIR}/qianqian/global_data_Qianqian/'
    f'1input_data/{year}global/era5land/'
)
OUT_DIR = (
    f'{ROOT_DIR}/qianqian/global_data_Qianqian/'
    f'1input_data/{year}global/era5land'
)

## Setup Dask cluster

NOTE: when working with NetCDF files (and the netcdf4 library) it is much better to work with many processes and few threads per process: netcdf4 can only read from one thread per process.

In [4]:
# cluster = LocalCluster(n_workers=16, threads_per_worker=1)

In [5]:
cluster = SLURMCluster(
    name='dask-worker',
    cores=16,
    processes=16,
    queue='fat',
    memory='120GiB',
    local_directory='$TMPDIR',
    walltime='3:00:00'
)

/home/qiahan/.conda/envs/mamba/envs/emulator/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36437 instead
  warnings.warn(


We use in total 4 * 16 = 64 single-threaded workers, and ~480 GiB total memory: 

In [6]:
cluster.scale(jobs=4)

In [7]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/36437/status,
Dashboard: /proxy/36437/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://145.136.63.1:43843,Workers: 0
Dashboard: /proxy/36437/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Converting the dataset 

In [4]:
era5_paths = sorted(
    glob.glob(f'{ERA5_DIR}/era5-land_*.nc')
)

In [5]:
len(era5_paths)

96

In [6]:
ds = xr.open_mfdataset(era5_paths, chunks={'longitude': 250, 'latitude': 250})
ds = ds.chunk({'time': 750})

The resulting dataset has 8 variables, with chunks of approximately 180 MiB:

In [7]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 3600, latitude: 1801, time: 8784)
Coordinates:
  * longitude  (longitude) float32 0.0 0.1 0.2 0.3 ... 359.6 359.7 359.8 359.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 2012-01-01 ... 2012-12-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    strd       (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(750, 250, 250), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-15 00:16:00 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [14]:
%%time
zarr_path = f'{OUT_DIR}/era5land.zarr'
ds.to_zarr(zarr_path, mode='w')

CPU times: user 26min 17s, sys: 1min 4s, total: 27min 22s
Wall time: 2h 16min 25s


When done with the conversion we switch off the cluster to release resources:

In [15]:
client.shutdown()

The overall Zarr archive is ~200 GB:

In [4]:
zarr_path = f'{OUT_DIR}/era5land.zarr'
!du -h $zarr_path

31G	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/sp
18G	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/ssrd
30G	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/v10
2.0K	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/longitude
2.0K	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/latitude
31G	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/d2m
11G	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/tp
29G	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/u10
2.0K	/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2012global/era5land/era5land.zarr/time
30G	/gpfs/work2/0/ttse0